<a href="https://colab.research.google.com/github/whatnews72/----/blob/master/mooc4%E1%84%8C%E1%85%AE%E1%84%8E%E1%85%A1_%E1%84%89%E1%85%B5%E1%86%AF%E1%84%89%E1%85%B3%E1%86%B82_RNN_LSTM_%E1%84%80%E1%85%A1%E1%86%B7%E1%84%89%E1%85%A5%E1%86%BC%E1%84%87%E1%85%AE%E1%86%AB%E1%84%89%E1%85%A5%E1%86%A8_%E1%84%89%E1%85%AE%E1%84%80%E1%85%A1%E1%86%BC%E1%84%89%E1%85%A2%E1%86%BC%E1%84%8B%E1%85%AD%E1%86%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 정의

- 문장의 감정 (긍정, 부정) 분류

# 데이터 세트

- 데이터 세트명 : 한국어 네이버 영화 리뷰
- 데이터 세트 출처 : https://github.com/e9t/nsmc

# 데이터 세트 상세 설명

- 각 파일은 id, document, label 세 개의 열로 구성됩니다.
- 열은 탭으로 구분됩니다(예: .tsv형식. 그러나 파일 확장자는 .txt초보자가 쉽게 접근할 수 있도록 함).
- 총 리뷰 20만개
  - ratings.txt: 전체 리뷰 20만개
  - ratings_test.txt: 테스트용 리뷰 50,000개
  - ratings_train.txt: 훈련용 리뷰 150,000개

- id: 리뷰 아이디, 네이버에서 제공
- document: 실제 리뷰
- label: 리뷰의 감정 클래스입니다. (0: 부정, 1: 긍정)


In [18]:
# 한글 형태소 분석 패키지 설치
!pip install kiwipiepy

In [19]:
# 패키지 로드
import numpy as np
import pandas as pd
import tensorflow as tf

# 한글 형태소 분석 패키지 로드
from kiwipiepy import Kiwi
kiwi = Kiwi()


In [20]:
# 훈련 데이터 불러오기 (부정 0, 긍정 1)
train_data = pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
train_data = train_data[:1000]
train_data.shape, display(train_data.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


((1000, 3), None)

In [21]:
# 검증 데이터 불러오기
test_data = pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')
test_data = test_data[:100]
test_data.shape, display(test_data.head())

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


((100, 3), None)

In [22]:
# 주요 전처리 (.....생략......)

In [23]:
# document 토큰화 전처리 - 1 (형태소 분석기 확인)
kiwi.tokenize('한글을 사랑합니다.')

[Token(form='한글', tag='NNG', start=0, len=2),
 Token(form='을', tag='JKO', start=2, len=1),
 Token(form='사랑', tag='NNG', start=4, len=2),
 Token(form='하', tag='XSV', start=6, len=1),
 Token(form='ᆸ니다', tag='EF', start=6, len=3),
 Token(form='.', tag='SF', start=9, len=1)]

In [24]:
# document 토큰화 전처리 - 2 (훈련 / 검증 데이터 토큰화)
x_train = []
for x in train_data['document']:
    tokens = [token[0] for token in kiwi.tokenize(x)] # 토큰화
    x_train.append(tokens)

x_test = []
for x in test_data['document']:
    tokens = [token[0] for token in kiwi.tokenize(x)] # 토큰화
    x_test.append(tokens)

In [25]:
# document 토큰화 전처리 - 3 (정수 인코딩)
# 훈련 데이터를 기준으로 각 토큰에 대해 정수 인코딩
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(x_train)

# 각 토큰 정수 값을 이용하여 모든 토큰을 숫자로 변경
x_train_ = tokenizer.texts_to_sequences(x_train)
x_test_ = tokenizer.texts_to_sequences(x_test)

# 50개 토큰으로 제한하여 줄이거나, 자르기
x_train_ = tf.keras.preprocessing.sequence.pad_sequences(x_train_, maxlen=50)
x_test_ = tf.keras.preprocessing.sequence.pad_sequences(x_test_, maxlen=50)

In [26]:
# 정답값 준비
y_train_ = train_data['label']
y_test_ = test_data['label']

In [27]:
#LSTM 모델 입력 데이터로 차원변환
x_train_ = x_train_.reshape(-1, 1, 50)
x_test_ = x_test_.reshape(-1, 1, 50)

# 모델링

In [28]:
# 모델 정의
i = tf.keras.Input(shape=(None, 50, ))
x = tf.keras.layers.LSTM(16, activation="relu")(i)
o = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=i, outputs=o, name='no3_model')

# 모델 학습 과정 설정
model.compile(loss='binary_crossentropy', optimizer='adamW', metrics=['accuracy'])

# 모델 학습
hist = model.fit(x_train_, y_train_, epochs=50, batch_size=64)


Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.5020 - loss: 53.7660
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4861 - loss: 37.7366 
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5273 - loss: 30.0141 
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5252 - loss: 23.4274 
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5358 - loss: 17.5718
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5537 - loss: 15.7698 
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5582 - loss: 14.3173 
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5579 - loss: 11.4813
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5434 - loss: 13.3857 
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5460 - loss: 10.6021 
Epoch 11/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5673 - loss: 9.0416 
Epoch 12/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - ac

# 모델 평가

In [29]:
# 모델 평가
score = model.evaluate(x_test_, y_test_)

print(x_test_.shape)
print(y_test_.shape)
print(score)

print("%s: %.2f%%" %(model.metrics_names[1], score[1]*100))

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.5431 - loss: 3.5404
(100, 1, 50)
(100,)
[4.112388610839844, 0.5400000214576721]
compile_metrics: 54.00%


In [30]:
# score 점수를 파일로 저장
fname = 'Week3_2.txt'
f = open(f'{fname}', 'w')
f.write(str(score[1]))
f.close()

# 예측

In [31]:
# 예측
pred = model.predict(x_test_)
test_data['pred'] = np.where(pred > 0.6, 1, 0)
test_data

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step


,id,document,label,pred
0,6270596,굳 ㅋ,1,0
1,9274899,GDNTOPCLASSINTHECLUB,0,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,0
...,...,...,...,...
95,6753658,전기톱은못들고다니는데 엔진톱이겠죠,0,0
96,9665771,완전 재밌엇는데 왜 평점이??,1,0
97,8757576,제임스 완이 내 목표임 ㄷ,1,0
98,9850643,1점고 아깝다. 개막장 영화의 원조라고나 할까.아내와 사별한 지 얼마나 지났다고 딴...,0,0


# evaluate() 이해

In [32]:
# model.evaluate(x_test_, y_test_) 이해
np.sum(test_data['label'] == test_data['pred']) / 100 * 100

54.0

# [실습 평가 방법]

본 프로그램을 실행한 후 생성되는 y_pred.csv 파일을 실습 평가 사이트에 등록하여 0.55 이상 획득 필요


In [33]:
# 모델 추론
y_pred_prob = model.predict(x_test_)

# 확률을 클래스 라벨로 변환 (0.5를 기준으로)
y_pred = (y_pred_prob > 0.5).astype(int)

# y_pred를 DataFrame으로 변환 후 CSV 파일로 저장
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.to_csv('y_pred_3_21.csv', index=False, header=False)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


<font size=5> [예측결과 제출](https://aifactory.space/task/2593/overview) </font>

# End Of Program